In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!apt-get install libatlas-base-dev
!pip install TA-Lib

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

measures_tickers = ['^GSPC', '^VIX']
all_symbols = stock_info_df_symbols#replace with list of stocks tickers

End_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
Start_date = (datetime.today() - timedelta(days=4*365)).strftime('%Y-%m-%d')
print(Start_date)
data = yf.download(all_symbols + measures_tickers , start=Start_date, end=End_date)
data.dropna(axis=0,inplace=True,how="all")
data.dropna(axis=1,inplace=True,how="all")

In [ ]:
#################### EXTRACT FROM YF INDUSTRY AND SECTOR FOR each stock
import yfinance as yf
import pandas as pd

# List of stock tickers
stock_tickers = all_symbols

# Create an empty DataFrame to store the information
stock_info_df = pd.DataFrame(columns=['Ticker', 'Industry', 'Sector'])

# Loop through each stock ticker
for ticker in stock_tickers:
    try:
        # Get information from Yahoo Finance
        stock = yf.Ticker(ticker)
        info = stock.info

        # Extract industry and sector information
        industry = info.get('industry', 'N/A')
        sector = info.get('sector', 'N/A')

        # Append information to the DataFrame
        stock_info_df = pd.concat([stock_info_df, pd.DataFrame({'Symbol': [ticker], 'Industry': [industry], 'Sector': [sector]})], ignore_index=True)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Save the DataFrame to a CSV file
stock_info_df.to_csv('/content/drive/MyDrive/stocks_data/stock_industrySector.csv', index=False)



In [ ]:
industry_dict = {industry: group['Symbol'].tolist() for industry, group in stock_info_df.groupby('Industry')}
sector_dict = {industry: group['Symbol'].tolist() for industry, group in stock_info_df.groupby('Sector')}
stock_info_df.set_index('Symbol', inplace=True)

In [ ]:
from numpy.ma.core import equal
import talib as tb
from sklearn.preprocessing import LabelEncoder


def preprocess_sentiment(df, start_time, end_time,measures_tickers,stocks_list,industry_dict,sector_dict,all_stocks_data):
     """
    Preprocesses stocks data to achive market sentimant

    Args:
        df (DataFrame): The input DataFrame containing sentiment data.
        start_time (str): The starting date for considering data in the preprocessing.
        end_time (str): The ending date for considering data in the preprocessing.
        measures_tickers (list): List of tickers for sentiment measures.
        stocks_list (list): List of stock tickers to consider.
        industry_dict (dict): Dictionary mapping stock tickers to industry names.
        sector_dict (dict): Dictionary mapping stock tickers to sector names.
        all_stocks_data (DataFrame): Additional information about all stocks.

    Returns:
        tuple: A tuple containing two elements - processed DataFrame (`df`) and a list of removed stock tickers (`removed_stocks`).

    The preprocess_sentiment function takes  data in the form of a DataFrame, along with
    additional parameters. It calculates various percentage changes and ratios
    based on the provided measures, groups, and frames.
    """
    df = df.copy()
    measures_tickers=['^GSPC']
    short_ind = 26
    #medium_ind = 50
    long_ind = 100

    # Define the intervals for weekly and monthly data
    frames = [short_ind, long_ind]
    group_dict = {}
    removed_stocks = []
    for i, stock in enumerate(stocks_list):
        if stock in measures_tickers or stock not in df['Adj Close'].columns:
          print('in',stock)
          removed_stocks.append(stock)
          continue
        Industry = all_stocks_data.loc[stock, 'Industry']
        #print(Industry,stock)
        Sector = all_stocks_data.loc[stock, 'Sector']
        groups = ['Industry']

        adj_close_stock = df['Adj Close', stock]
        volume_stock = df[f'Volume'][stock]

        for frame in frames:
            df[f'pct_change_{frame}', stock] = adj_close_stock.pct_change(periods=frame)

            for measure in measures_tickers:#measures calc
                measure_adj_close = df['Adj Close'][measure]

                df[f'pct_change_{frame}_{measure}', stock] = measure_adj_close.pct_change(periods=frame)

            for group in groups:#groups calc
                if group == 'Industry':
                    group_list = industry_dict[Industry]
                else:
                    group_list = sector_dict[Sector]
                #group_list = ['A','AA'] #need to change
                if (group,frame) in group_dict.keys():
                    df[f'pct_change_{frame}_{group}', stock] = group_dict[(group,frame)]
                else:
                    df[f'pct_change_{frame}_{group}', stock] = df['Adj Close'][group_list].pct_change(periods=frame).mean(axis=1, )
                    group_dict[(group,frame)] = list(df[f'pct_change_{frame}_{group}', stock])

            #for group in groups:#ratios with the stock and indexes
              #  df[f'ratio_pct_change_{frame}_{group}_{stock}', stock] = df[f'pct_change_{frame}_Industry', stock] / df[f'pct_change_{frame}_Sector', stock]

            for measure in measures_tickers: #measures & indexses & stock with measures ratios
              if measure == '^VIX':
                continue

              df[f'ratio_pct_change_{frame}_{measure}', stock] = df[f'pct_change_{frame}', stock] / df[f'pct_change_{frame}_{measure}', stock]

              for group in groups:
                  df[f'ratio_pct_change_{frame}_{measure}_{group}', stock] = df[f'pct_change_{frame}_{group}', stock] / df[f'pct_change_{frame}_{measure}', stock]
        if i%100 == 0:
          print(i,'finished')
    return df,removed_stocks

sentimant_df,removed_stocks = preprocess_sentiment(df=data, start_time=Start_date ,end_time=End_date,measures_tickers=measures_tickers,stocks_list=all_symbols,industry_dict=industry_dict,sector_dict=sector_dict,all_stocks_data=stock_info_df)
for s in removed_stocks:
    all_symbols.remove(s)
len(all_symbols)

In [ ]:
import talib as tb
from sklearn.preprocessing import LabelEncoder
import hashlib

def preprocess_technical_labels(df = pd.DataFrame(), start_time = '2017-01-01', end_time = '2023-12-01',measures_tickers=[],stocks_list=[],all_stocks_data=pd.DataFrame()):
    """
    Preprocesses technical indicators for machine learning.

    Args:
        df (DataFrame): The input DataFrame containing stock data.
        stocks_list (list): List of stock tickers to consider.
        all_stocks_data (DataFrame): Additional information about all stocks.

    Returns:
        DataFrame: Processed DataFrame with added technical indicators.

    The preprocess_technical_indicators function takes stock data in the form of a DataFrame ),
    along with a list of stock tickers and additional stock information.
    It calculates various technical indicators such as Exponential Moving Averages (EMA), Simple Moving Averages (SMA),
    Directional Movement Index (ADX), Average Directional Index (ADI), and more for each stock in the provided list.

    """
    df = df.copy()
    df.columns = df.columns.swaplevel(0, 1) #fix columns name

    short_ind = 26
    medium_ind = 50
    long_ind = 200

    # Define the intervals for weekly and monthly data
    weekly_interval = 5
    monthly_interval = 26

    # Define the bin edges for the present change groups
    bin_edges = [-10, -0.1, -0.05, -0.025, 0, 0.025, 0.05, 0.1, 10]
    # Define labels for the present change groups
    labels = [ 0, 1, 2, 3, 4, 5, 6, 7]

    for i, stock in enumerate(stocks_list):

        adj_close_stock = df[stock,'Adj Close']
        High_stock = df[stock,'High']
        Low_stock = df[stock,'Low']
        Volume_stock = df[stock,'Volume']
        open_Stock = df[stock,'Open']
        df[stock, "ema_short"] = tb.EMA(adj_close_stock, timeperiod=short_ind)
        df[stock, "ema_medium"] = tb.EMA(adj_close_stock, timeperiod=medium_ind)
        df[stock, "ema_long"] = tb.EMA(adj_close_stock, timeperiod=long_ind)

        #continue if thers was no high jump monthly in either direction
        month_labels = pd.cut(adj_close_stock.pct_change(periods=monthly_interval), bins=bin_edges, labels=labels, right=False)

        # Add a new column 'present_change_group' with labels for present change groups
        df[stock, 'label_day'] = pd.cut(adj_close_stock.pct_change(), bins=bin_edges, labels=labels, right=False)
        df[stock, 'label_week'] = pd.cut(adj_close_stock.pct_change(periods=weekly_interval), bins=bin_edges, labels=labels, right=False)
        df[stock, 'label_month'] = month_labels

        df[stock, 'pct_change_week'] = adj_close_stock.pct_change(periods=weekly_interval)
        df[stock, 'pct_change_month'] = adj_close_stock.pct_change(periods=monthly_interval)


        df[stock, 'Industry'] = all_stocks_data.loc[stock, 'Industry' ]
        df[stock, 'Sector'] = all_stocks_data.loc[stock, 'Sector' ]
        df[stock, 'stock'] = stock

      #Technical indicators
        # OVERLAP INDICATORS


        df[stock, "dema_short"] = tb.DEMA(adj_close_stock, timeperiod=short_ind)
        df[stock, "kama_short"] = tb.KAMA(adj_close_stock, timeperiod=short_ind)
        df[stock, "dema_medium"] = tb.DEMA(adj_close_stock, timeperiod=medium_ind)
        df[stock, "kama_medium"] = tb.KAMA(adj_close_stock, timeperiod=medium_ind)
        df[stock, "dema_long"] = tb.DEMA(adj_close_stock, timeperiod=long_ind)
        df[stock, "kama_long"] = tb.KAMA(adj_close_stock, timeperiod=long_ind)

        df[stock, "sma_short"] = tb.SMA(adj_close_stock, timeperiod=short_ind)
        df[stock, "sma_medium"] = tb.SMA(adj_close_stock, timeperiod=medium_ind)
        df[stock, "sma_long"] = tb.SMA(adj_close_stock, timeperiod=long_ind)
        df[stock, "sar"] = tb.SAR(High_stock, Low_stock)

        # MOMENTUM INDICATORS
        df[stock, "adx_short"] = tb.ADX(High_stock, Low_stock, adj_close_stock, timeperiod=short_ind)
        df[stock, "adx_medium"] = tb.ADX(High_stock, Low_stock, adj_close_stock, timeperiod=medium_ind)
        df[stock, "adx_long"] = tb.ADX(High_stock, Low_stock, adj_close_stock, timeperiod=long_ind)

        df[stock, "apo_short"] = tb.APO(adj_close_stock, fastperiod=long_ind, slowperiod=short_ind)
        df[stock, "apo_medium"] = tb.APO(adj_close_stock, fastperiod=long_ind, slowperiod=medium_ind)
        df[stock, "apo_long"] = tb.APO(adj_close_stock, fastperiod=long_ind, slowperiod=long_ind)

        df[stock, "cci_short"] = tb.CCI(High_stock, Low_stock, adj_close_stock, timeperiod=short_ind)
        df[stock, "cci_medium"] = tb.CCI(High_stock, Low_stock, adj_close_stock, timeperiod=medium_ind)
        df[stock, "cci_long"] = tb.CCI(High_stock, Low_stock, adj_close_stock, timeperiod=long_ind)

        df[stock, "bop"] = tb.BOP(open_Stock, High_stock, Low_stock, adj_close_stock)
        df[stock, "macd"], df[stock,"macdsignal"], df[stock,"macdhist"] = tb.MACD(adj_close_stock, fastperiod=12, slowperiod=26, signalperiod=9)

        df[stock, "mfi_short"] = tb.MFI(High_stock, Low_stock, adj_close_stock, Volume_stock, timeperiod=short_ind)
        df[stock, "mom_short"] = tb.MOM(adj_close_stock, timeperiod=short_ind)
        df[stock, "mfi_medium"] = tb.MFI(High_stock, Low_stock, adj_close_stock, Volume_stock, timeperiod=medium_ind)
        df[stock, "mom_medium"] = tb.MOM(adj_close_stock, timeperiod=medium_ind)
        df[stock, "mfi_long"] = tb.MFI(High_stock, Low_stock, adj_close_stock, Volume_stock, timeperiod=long_ind)
        df[stock, "mom_long"] = tb.MOM(adj_close_stock, timeperiod=long_ind)

        df[stock, "rsi_short"] = tb.RSI(adj_close_stock, timeperiod=short_ind)
        df[stock, "rsi_medium"] = tb.RSI(adj_close_stock, timeperiod=medium_ind)
        df[stock, "rsi_long"] = tb.RSI(adj_close_stock, timeperiod=long_ind)

        # VOLUME INDICATORS
        df[stock, "ad"] = tb.AD(High_stock, Low_stock, adj_close_stock, Volume_stock)
        df[stock, "adosc_short"] = tb.ADOSC(High_stock, Low_stock, adj_close_stock, Volume_stock, fastperiod=short_ind, slowperiod=short_ind)
        df[stock, "adosc_medium"] = tb.ADOSC(High_stock, Low_stock, adj_close_stock, Volume_stock, fastperiod=short_ind, slowperiod=medium_ind)
        df[stock, "adosc_long"] = tb.ADOSC(High_stock, Low_stock, adj_close_stock, Volume_stock, fastperiod=short_ind, slowperiod=long_ind)

        df[stock, "obv"] = tb.OBV(adj_close_stock, Volume_stock)
        df[stock, "trange"] = tb.TRANGE(High_stock, Low_stock, adj_close_stock)
        df[stock, "atr_short"] = tb.ATR(High_stock, Low_stock, adj_close_stock, timeperiod=short_ind)
        df[stock, "atr_medium"] = tb.ATR(High_stock, Low_stock, adj_close_stock, timeperiod=medium_ind)
        df[stock, "atr_long"] = tb.ATR(High_stock, Low_stock, adj_close_stock, timeperiod=long_ind)

        df.index = pd.to_datetime(df.index)

        # Calculate 52-week high and low
        df['year'] = df.index.year
        df['month'] = df.index.month
        df['day'] = df.index.day

        if i%100 == 0:
          print(i,'finished')
    return df

sentimant_df.dropna(axis=0,inplace=True,how="all")
sentimant_df.dropna(axis=1,inplace=True,how="all")
ta_sentimant_df_new = preprocess_technical_labels(df=sentimant_df,measures_tickers=measures_tickers,stocks_list=all_symbols,all_stocks_data=stock_info_df)


0 finished


<ipython-input-7-8e180421cb9f>:129: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[stock, "atr_short"] = tb.ATR(High_stock, Low_stock, adj_close_stock, timeperiod=short_ind)
<ipython-input-7-8e180421cb9f>:130: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[stock, "atr_medium"] = tb.ATR(High_stock, Low_stock, adj_close_stock, timeperiod=medium_ind)
<ipython-input-7-8e180421cb9f>:131: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

100 finished
200 finished
300 finished
400 finished
500 finished
600 finished
700 finished
800 finished
900 finished
1000 finished
1100 finished
1200 finished
1300 finished
1400 finished
1500 finished
1600 finished
1700 finished
1800 finished
1900 finished
2000 finished
2100 finished
2200 finished
2300 finished
2400 finished
2500 finished
2600 finished
2700 finished
2800 finished
2900 finished
3000 finished
3100 finished
3200 finished
3300 finished
3400 finished
3500 finished
3600 finished
3700 finished
3800 finished
3900 finished
4000 finished
4100 finished
4200 finished
4300 finished


In [ ]:
import numpy as np
def t_df_labels(label_type = 'label_week',df = pd.DataFrame(),stocks_list = []):# Shifts back the labels for each time lapce
    df = df.copy()
    df_working = pd.DataFrame()
    stocks_number = len(stocks_list)
    count = 0
    df_nan_values_day = pd.DataFrame({'label_day': [np.nan] })
    df_nan_values_week = pd.DataFrame({label_type: [np.nan] * 5})
    df_nan_values_month = pd.DataFrame({'label_month': [np.nan] * 20})
    date_col = df.index

    for i,stock in enumerate(stocks_list):
      stock_df = df[stock]
      stock_df['Symbol'] = stock
      stock_df['date'] = date_col
      if label_type in stock_df.columns:
          df_day = pd.concat([pd.DataFrame({'label_day': stock_df['label_day'][1:] }), df_nan_values_day], ignore_index=True)
          df_day.index = stock_df.index
          stock_df['label_day'] = df_day

          df_week = pd.concat([pd.DataFrame({label_type: stock_df[label_type][5:] }), df_nan_values_week], ignore_index=True)
          df_week.index = stock_df.index
          stock_df[label_type] = df_week
          df_week_pct_change = pd.concat([pd.DataFrame({'pct_change_week': stock_df['pct_change_week'][5:] }), df_nan_values_week], ignore_index=True)
          df_week_pct_change.index = stock_df.index
          stock_df['pct_change_week'] = df_week_pct_change['pct_change_week']

          df_month = pd.concat([pd.DataFrame({'label_month': stock_df['label_month'][20:] }), df_nan_values_month], ignore_index=True)
          df_month.index = stock_df.index
          stock_df['label_month'] = df_month
          df_month_pct_change = pd.concat([pd.DataFrame({'pct_change_month': stock_df['pct_change_month'][20:] }), df_nan_values_month], ignore_index=True)
          df_month_pct_change.index = stock_df.index
          stock_df['pct_change_month'] = df_month_pct_change['pct_change_month']

          stock_df.reset_index(drop=True, inplace=True)
          df_working = pd.concat([df_working, stock_df], ignore_index=True)
      else:
          count+=1
    print(count,'dont have label',df_working)
    return df_working


In [ ]:
working_df_updated_new = t_df_labels(label_type = 'label_week',df = ta_sentimant_df_new,stocks_list = all_symbols)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Initialize a LabelEncoder
columns_drop_withNan = ['Adj Close',"ema_long","mom_long"]
working_df_updated_new1 =  working_df_updated_new.dropna(subset=columns_drop_withNan)
label_encoder = LabelEncoder()

# Apply Label Encoding to each column
working_df_updated_new1['Industry_encoded'] = label_encoder.fit_transform(working_df_updated_new1['Industry'])#encode strings features
working_df_updated_new1['Sector_encoded'] = label_encoder.fit_transform(working_df_updated_new1['Sector'])
working_df_updated_new1.to_csv('/content/drive/MyDrive/stocks_data/Consecutive_stocks.csv', index=False)